# <span style="color:#1f77b4">**Data Analytics 04 - Visualization**</span>

This notebook uses Delta Live Tables SQL to create raw and aggregated tables, then views results.



### <span style="color:#1f77b4">**Storing CSV file to DBFS (Databricks File System)**</span>

Create a DBFS folder and download the product CSV.

- `%sh` runs shell commands in the notebook context.
- `wget` downloads the CSV into DBFS.



In [ ]:
# Create a DBFS folder and download sample CSVs
%sh
rm -r /dbfs/delta_lab
mkdir /dbfs/delta_lab
wget -O /dbfs/delta_lab/products.csv https://raw.githubusercontent.com/Ch3rry-Pi3-Azure/DataBricks-Data-Analytics/refs/heads/main/data/products.csv

### <span style="color:#1f77b4">**Creating Delta Live Tables using SQL**</span>

Define DLT tables using SQL for streaming-style pipelines.

- `CREATE OR REFRESH LIVE TABLE` defines a DLT table.



#### <span style="color:#1f77b4">**Loading the raw dataset in the very first layer**</span>

Create the raw DLT table from the CSV source.

- `read_files` reads from DBFS using SQL.



In [ ]:
CREATE OR REFRESH LIVE TABLE raw_products_data
COMMENT "Products sample dataset"
AS
SELECT
 ProductID,
 ProductName,
 Category,
 ListPrice
FROM read_files('dbfs:/delta_lab/products.csv', format => 'csv', header => true)

#### <span style="color:#1f77b4">**Generating Business Aggregates from the raw layer dataset**</span>

Aggregate the raw table into category totals.

- `sum` computes totals per category.
- `GROUP BY` creates the aggregation.



In [ ]:
CREATE OR REFRESH LIVE TABLE aggregated_products_data
COMMENT "Aggregated data"
AS
SELECT
   Category,
   sum(ListPrice) as Total_Price
FROM live.raw_products_data
GROUP BY Category;

### <span style="color:#1f77b4">**Viewing the final results**</span>

Load the aggregated Delta table and display the results.

- `spark.read.format("delta")` reads the DLT output.
- `display` renders the table.



In [ ]:
df = spark.read.format("delta").load('/pipelines/delta_lab/tables/aggregated_products_data')
display(df)